<a href="https://colab.research.google.com/github/rtldr/DeepDecompiler/blob/master/DeepDecompiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install pyelftools

     |████████████████████████████████| 153kB 5.8MB/s 


In [5]:
# DATA_FOLDER_PATH = 'drive/MyDrive/data'
# Upload the tar.gz data files to the VM and tar -xzvf them
DATA_V2_FOLDER_PATH = 'data_v2'

In [7]:
# Simple asm tokenizer - TODO maybe look into better ways of doing this
from elftools.elf.elffile import ELFFile

NUM_TOKENS = 2**8 + 2
EOF = 256
PAD = 257

def tokenize_asm_file(filename):
    with open(filename, "rb") as f:
        elf_file = ELFFile(f)
        for section in elf_file.iter_sections():
            if section.name == '.text':
                bytes_list = list(section.data())
    # Add EOF
    bytes_list.append(EOF)
    return bytes_list
    
if __name__ == "__main__":
    m = 0
    count = 0
    for i in range (1024):
        tokenized = tokenize_asm_file(DATA_V2_FOLDER_PATH + "/asm/" + str(i))
        if len(tokenized) < 100:
            count += 1
            m = max(m, len(tokenized))
    print(m)
    print(count)


99
756


In [9]:
# C Tokenizer

from pycparser.c_lexer import CLexer

# C token space
# 0 - 99 tokens generated by CLexer
# 100 - 227 printable ASCII characters (100 -> Ascii 32) TODO - switch to byte pair encodings
# 228 - END_ID_CONST
# 229 - END_LITERAL
# 230 - EOF
# 231 - SOS 
EOF = 100
SOS = 101
PAD = 102
NUM_TOKENS = 103
# EOF = 230
# SOS = 231
# NUM_TOKENS = 232

class CTokenizer:
    def __init__(self):
        self.lexer = CLexer(self._on_error, self._on_left_brace, self._on_left_brace, self._type_lookup_func)
        self.lexer.build(optimize=True, lextab='pycparser.lextab', outputdir='')
        self.semicolon_token = self.lexer.tokens.index('SEMI')

    # Lexer Functions
    def _on_error(self, msg, line, column):
        print("Error tokenizing:", msg)

    def _on_left_brace(self):
        pass

    def _on_right_brace(self):
        pass

    def _type_lookup_func(self, name):
        return False

    def tokenize_file(self, filename, include_SOS=False):
        with open(filename) as f:
            # Skip first line of file
            f.readline()
            text = f.read()
        tokens = []
        self.lexer.input(text)
        t = self.lexer.token()
        while t is not None:
            tokens += self._process_lexer_token(t)
            t = self.lexer.token()
        tokens.append(EOF)

        # Strip header info by removing everything before first two semicolons
        tokens = tokens[tokens.index(self.semicolon_token) + 1:]
        tokens = tokens[tokens.index(self.semicolon_token) + 1:]

        return [SOS] + tokens if include_SOS else tokens 

    def _process_lexer_token(self, ltoken):
        ltype = ltoken.type
        token = [self.lexer.tokens.index(ltype)]

        # if "LITERAL" in ltype:
        #     value = ltoken.value[1:-1]
        #     for char in value:
        #         token.append(ord(char) + 100 - 32)
        #     token.append(229)
        # elif ("CONST" in ltype and ltype != "CONST") or ltype == "ID":
        #     value = ltoken.value
        #     for char in value:
        #         token.append(ord(char) + 100 - 32)
        #     token.append(228)

        return token

    def tokens_to_string(self, tokens):
        result = ""
        for token in tokens:
            if 0 <= token < 100:
                result += " " + self.lexer.tokens[token]
            elif 100 <= token < 228:
                result += chr(token - 100 + 32)
            elif token == 228:
                result += "END_ID_CONST"
            elif token == 229:
                result += "END_STR_LITERAL"
            elif token == EOF:
                result += " EOF"
            else:
                result += " UNKNOWN_TOKEN"

        return result[1:]
        
    def tokens_to_C_string(self, tokens):
        C_dict = {
            "LPAREN" : '(',
            "RPAREN" : ')',
            # Operators
            "PLUS" : '+',
            "MINUS" : '-',
            "TIMES" : '*',
            "DIVIDE" : '/',
            "MODULO" : '%',
            "OR" : '|',
            "AND" : '&',
            "NOT" : '~',
            "XOR" : '^',
            "LSHIFT" : '<<',
            "RSHIFT" : '>>',
            "LOR" : '||',
            "LAND" : '&&',
            "LNOT" : '!',
            "LT" : '<',
            "GT" : '>',
            "LE" : '<=',
            "GE" : '>=',
            "EQ" : '==',
            "NE" : '!=', 
            # Assignment Operators
            "EQUALS" : '=',
            "TIMESEQUAL" : '*=',
            "DIVEQUAL" : '=',
            "MODEQUAL" : '=',
            "PLUSEQUAL" : '+=',
            "MINUSEQUAL" : '-=',
            "LSHIFTEQUAL" : '<<=',
            "RSHIFTEQUAL" : '>>=',
            "ANDEQUAL" : '&=',
            "OREQUAL" : '|=',
            "XOREQUAL" : '^=',
            # Increment/decrement
            "PLUSPLUS" : '++',
            "MINUSMINUS" : '--',
            # ->
            "ARROW" : '->',
            # ?
            "TERNARY" : '?',
            # Delimeters
            "LPAREN" : '(',
            "RPAREN" : ')',
            "LBRACKET" : '[',
            "RBRACKET" : ']',
            "BRACE" : '{',
            "RBRACE" : '}',
            "COMMA" : ',',
            "PERIOD" : '.',
            "SEMI" : ';',
            "COLON" : ':',
            "ELLIPSIS" : '...',
            
        }
        result = ""
        for_semi = 0
        indent = 0; 
        tab = "    "
        for token in tokens:
            if 0 <= token < 100:
                token_str = self.lexer.tokens[token]
                if token_str == "LBRACE":
                    result += "\n" + indent*tab
                    indent += 1
                    result += "{\n" + indent*tab
                elif token_str == "RBRACE":
                    indent -= 1
                    result += "\n" + indent*tab
                    result += "}\n" + indent*tab
                elif token_str == "FOR":
                    result += "for"
                    for_semi = 2
                elif token_str == "SEMI":
                    result += ";"
                    if for_semi > 0:
                        for_semi -= 1
                    else: 
                        result += "\n" + indent*tab
                elif token_str in C_dict: 
                    result += C_dict[token_str]
                else: 
                    result += " " + self.lexer.tokens[token].lower()
            elif 100 <= token < 228:
                 result += chr(token - 100 + 32)
            elif token == 228:
                result += "end_id_const"
            elif token == 229:
                result += "end_str_literal"
            elif token == EOF:
                result += " eof"
            else:
                result += " unknown_token"

        return result[1:]


if __name__ == "__main__":
    tkn = CTokenizer()
    tokens = tkn.tokenize_file(DATA_V2_FOLDER_PATH + "/c/0.c")
    print(tkn.tokens_to_C_string(tokens))


static id id( void)
{
     id id= int_const_hex;
     return id;
    
}
 


static id id( void)
{
     id id= int_const_hex;
     return id;
    
}
 


In [21]:
tokenized = tokenize_asm_file(DATA_V2_FOLDER_PATH + "/asm/" + '0')
print(tokenized)

[243, 15, 30, 250, 85, 72, 137, 229, 199, 69, 252, 159, 4, 26, 233, 139, 69, 252, 72, 152, 93, 195, 100]


In [22]:
from fastai import *
from fastai.text import *

In [29]:
# Create C data loader
c_tokenizer = CTokenizer()
loaders = []
for i in range(0):
  tokens = c_tokenizer.tokenize_file(DATA_V2_FOLDER_PATH + '/c/' + str(i) + '.c')
  dl = DataLoader(tokens)
  loaders.append(dl)

tdb = TextDataBunch.from_tokens()
# Use ASM numbers as tokens for the model
# Use C numbers as tokens for the model

# Model thinks numbers are actual words.
# Model trains on C numbers, spits out a C number. We convert all these numbers to C strings to see what the model is spitting out
